# data_generator
This repository contains the code for generating the dataset in npz format for the Capstone project.

### Instructions
1. Create folders for each class in the same directory, rename each folder as its label name. (It's what annotation_parser can do)
2. Put images to each folder according to its label.
3. set scale_percent(resizing image to scale_percent% of original image, unit:%, default = 100%) 
4. write down all labels (all folder's name) in array class_name
5. set sliding window size to the correct width of the image, note that length must be equal to width.
6. Adjust path, image format and export file name

### Reference
1. numpy save data: https://numpy.org/doc/stable/reference/generated/numpy.savez.html
2. Iris datset training sample in Bunch format. https://scikit-learn.org/stable/auto_examples/svm/plot_iris_svc.html

In [ ]:
import cv2
def resizeImage(image, scale_percent):
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    return resized

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import image as im
import numpy as np
import os
from sklearn.utils import Bunch
scale_percent = 100
image_count_s = []
total_image_count = 0
s = ['1', '5']
label_type = [1, 0]
image_count_s = [1415, 1415 * 5]
count = 1415 * 6
total_image_count += count

# sliding window parameters set up
sliding_window_size = 200
data = np.zeros([total_image_count, int(sliding_window_size*scale_percent*0.01), int(sliding_window_size*scale_percent*0.01)])
depth = np.zeros(total_image_count)
labels = np.zeros(total_image_count)
count = 0

for i, image_index in enumerate(s):
    path = f'unprocessed_images/cropped/200_40/test_1st2ndDataset/{image_index}/' # adjust to the right path of your dataset
    if not os.path.exists(path):
        continue

    image_count = 0
    images = []
    if i == 0:
        for file_name in os.listdir(path):
            if file_name.endswith(".jpg"): # adjust to the right format of your dataset
                image = im.imread(f'{path}{file_name}')
                #trans into grayscale
                rgb_weights = [0.2989, 0.5870, 0.1140]
                image = np.dot(image[...,:3], rgb_weights)
                images.append(resizeImage(image, scale_percent))
                depth[count + image_count] = int(file_name.split("_")[0])
                image_count += 1
    else:
        n = len(os.listdir(path))
        random_image_index = set(np.random.randint(n, size=image_count_s[i]))
        for j, file_name in enumerate(os.listdir(path)):
            if j in random_image_index and file_name.endswith(".jpg"): # adjust to the right format of your dataset
                image = im.imread(f'{path}{file_name}')
                #trans into grayscale
                rgb_weights = [0.2989, 0.5870, 0.1140]
                image = np.dot(image[...,:3], rgb_weights)
                images.append(resizeImage(image, scale_percent))
                depth[count + image_count] = int(file_name.split("_")[0])
                image_count += 1 
        
    data[count:count + image_count] = np.array(images)
    labels[count:count + image_count] = np.ones(len(images)) * label_type[i]
    count += image_count 

data = data.astype(np.uint8)
depth = depth.astype(np.uint16)
labels = labels.astype(np.uint8)

np.savez(str(total_image_count)+'_datasetA.npz', data=data, depth=depth, label=labels) # adjust to the desired name of your output dataset

In [ ]:
# check the dataset built successfully
npdata = np.load(f'3306_dataset.npz')
depth = npdata['depth']
label = npdata['label']
print(depth.shape)
print(label.shape)